Notebook for doing inference only with an ensemble -- experiments are conducted elsewhere. Model is as stated; `MaxAbsScaler` and `SelectKBest(k=80)` seem to be the best options as of 20210824. (Though model hyperparams haven't been fine-tuned with the scaler and the feature selector as of yet.) **And they should be, since performance is down!**


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# general ML tooling
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
import wandb
from wandb.xgboost import wandb_callback
# import timm
from pathlib import Path
import os
import math
import seaborn as sns
from datetime import datetime

from xgboost import XGBRegressor
# from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MaxAbsScaler, StandardScaler, MinMaxScaler, RobustScaler, PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_regression
from joblib import dump, load
# feature engineering tools
# from sklearn.feature_selection import mutual_info_regression
# import featuretools as ft


In [2]:
%matplotlib inline
%config Completer.use_jedi = False
os.environ['WANDB_NOTEBOOK_NAME'] = 'XGBoost_hyperparameter_sweep_20210827.ipynb'

# defining default config here; n_estimators, learning_rate, scaler, reg_alpha, and reg_lambda will be the variables
config_defaults = {
    # model config
    "wandb": True,
    "model":XGBRegressor,
    "tree_method": "auto", # set to 'gpu_hist' to try GPU if available
    "booster": 'gbtree', # dart may be marginally better, but will opt for this quicker approach as a default
    "n_estimators": 600, 
    "max_depth": 3,
    "learning_rate": 0.1,
    "test_size": 0.2,
    "scaler": MaxAbsScaler,
#     "feature_selector": SelectKBest,
#     "k_best": 80,
#     "feature_selection_scoring": f_regression,
    'random_state': 42,
    'subsample': 1,
    'n_jobs': -1,
    'verbosity': 1,
    'k_folds': 1,
    'feature_creator': PolynomialFeatures,
    'features_selection': 'manual_by_importance',
}

config_run = {
    # wandb config:
    'name': f"{os.environ['WANDB_NOTEBOOK_NAME'][:-6]}_{datetime.now().strftime('%H%M%S')}", # just removes the .ipynb extension, leaving the notebook filename's stem
    'tags': ['XGBoost', 'kfold', 'scaling', 'sweep'],
    'notes': "Second major sweep for hyperparameters",
}

In [3]:
# second tree sweep
sweep_config = {
    "method": "bayes", # try grid or random
    "metric": {
      "name": "rmse",
      "goal": "minimize"   
    },
    "parameters": {
        "max_depth": {
            "distribution": "int_uniform", 
            "min": 2,
            "max": 4,
        },
        "learning_rate": {
            "distribution": "uniform", 
            "min": 0.001,
            "max": 0.12
            #             "values": [0.1, 0.2, 0.3]
        },
        "n_estimators": {
            "distribution": "int_uniform",
            "min": 500,
            "max": 3500,
#             "values": [50, 125, 200]
        },
#         "scaler": {
#             "distribution": "constant",
#             "values": [MaxAbsScaler, StandardScaler, MinMaxScaler, RobustScaler],
#         },
        "reg_alpha": {
            "distribution": "uniform",
            "min": 1,
            "max": 3,
        },
        "reg_lambda": {
            "distribution": "uniform",
            "min": 2,
            "max": 5,
        },
        "subsample": {
            "distribution": "uniform",
            "min": 0.75,
            "max": 1,
        }
    }
}

In [4]:
# datapath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/202108_august/')

In [5]:
# df = pd.read_csv(datapath/'train.csv', index_col='id', low_memory=False)
# df.index.name = None
# df.to_feather(path='./dataset_df.feather')

# load unaltered dataset
# df = pd.read_feather(path='dataset_df.feather')
# df.index.name = 'id'

In [6]:
# y = df.loss

In [7]:
# load all the polynomialfeatures generated with `PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)`
# X_np = np.load(datapath/'X_poly_unscaled.npy')
# X = pd.DataFrame(X_np)

In [8]:
# X.columns

In [9]:
# # prep features from unaltered dataset
# features = [x for x in df.columns if x != 'loss']
# X = df[features]

# Feature Creation

In [10]:
# poly = PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)
# X_poly = poly.fit_transform(X)

In [11]:
# X_poly_names = poly.get_feature_names(X.columns)

In [12]:
# X = pd.DataFrame(X_poly, columns=X_poly_names)

# Scaling
Now, going to scale using `MaxAbsScaler`

In [13]:
# scaler = config['scaler']()
# X = scaler.fit_transform(X)

# Training Function

In [14]:
# datapath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/202108_august/')
# df = pd.read_feather(path='dataset_df.feather')
# df.index.name = 'id'
# y = df.loss
# features = [x for x in df.columns if x != 'loss']
# X = df[features]
# poly = PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)
# X_poly = poly.fit_transform(X)
# X_poly_names = poly.get_feature_names(X.columns)
# X = pd.DataFrame(X_poly, columns=X_poly_names)
# scaler = MaxAbsScaler()
# X_scaled = scaler.fit_transform(X)
# features = pd.read_csv('X_candidates_20210827.csv').columns
# print(len(features))
# X_scaled_df = pd.DataFrame(X_scaled, columns=X_poly_names)

In [15]:
# features[1:]

In [16]:
# X_final = X_scaled_df[features[1:]]

In [17]:
# X_final.head()

In [18]:
# X_scaled_df.loc[:, 'f25'].head()

In [19]:
# X.columns

In [20]:
def train_sweep(): #X_train, X_valid, y_train, y_valid, config):#, scaler): # passed in via config dict for now
    """
    Basic training function. Note that some of the options passed via the argument are
    in fact hard-coded in, to avoid inconveniences.
    :param config: dict with things to be logged in WandB, some to be used in function
    """
    
    
#     if config['wandb']:
    wandb.init(config=config_defaults)   
    config = wandb.config
    datapath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/202108_august/')
    df = pd.read_feather(path='dataset_df.feather')
    df.index.name = 'id'
    y = df.loss
    features = [x for x in df.columns if x != 'loss']
    X = df[features]
    poly = PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)
    X_poly = poly.fit_transform(X)
    X_poly_names = poly.get_feature_names(X.columns)
    X = pd.DataFrame(X_poly, columns=X_poly_names)
    scaler = MaxAbsScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled_df = pd.DataFrame(X_scaled, columns=X_poly_names)
    features = pd.read_csv('X_candidates_20210827.csv').columns
    X = X_scaled_df[features[1:]]
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                          test_size=config['test_size'], 
                                                          random_state=config['random_state']
                                                         )
    
    
    # selecting features
#     selector = config['feature_selector'](score_func=config["feature_selection_scoring"], 
#                                           k=config['k_best'])
#     X_train_fs = selector.fit_transform(X_train_s, y_train)
#     X_valid_fs = X_valid_s[:, selector.get_support()] # ensures same features are used in validation

    # split the dataset
    model = XGBRegressor(
        tree_method=config['tree_method'],
        booster=config['booster'],
        n_estimators=config['n_estimators'], 
        max_depth=config['max_depth'],
        learning_rate=config['learning_rate'], 
        reg_alpha=config['reg_alpha'],
        reg_lambda=config['reg_lambda'],
#         test_size=config['test_size'],
        subsample=config['subsample'],
        random_state=config['random_state'],
        n_jobs=config['n_jobs'], 
        verbosity=config['verbosity'], 
    )
#     wandb.log({'params': model.get_params()}) # logging model parameters
#     if config['wandb']:
    model.fit(X_train, y_train, callbacks=[wandb.xgboost.wandb_callback()])
    y_preds = model.predict(X_valid)
    mse = mean_squared_error(y_valid, y_preds)
    rmse = math.sqrt(abs(mse))
#     print(f"MSE is {mse}\nRMSE is {rmse}")   
#     if config['wandb']:
    wandb.log({'mse':mse, 'rmse':rmse})
#     wandb.finish()   
#     return model
    

# Hyperparameter Sweep

In [21]:
sweep_id = wandb.sweep(sweep_config, project="202108_Kaggle_tabular_playground")

Create sweep with ID: 4tk3ryjn
Sweep URL: https://wandb.ai/hushifang/202108_Kaggle_tabular_playground/sweeps/4tk3ryjn


In [22]:
wandb.agent(sweep_id, train_sweep)

wandb: Agent Starting Run: 2pg4mss2 with config:
wandb: 	learning_rate: 0.03249228600416143
wandb: 	max_depth: 4
wandb: 	n_estimators: 3323
wandb: 	reg_alpha: 1.6331976164011603
wandb: 	reg_lambda: 3.0763504519700042
wandb: 	subsample: 0.9775705002614221
wandb: Currently logged in as: hushifang (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/xgboost/training.py:17: UserWarning: Old style callback is deprecated.  See: https://xgboost.readthedocs.io/en/latest/python/callbacks.html
  warnings.warn(f'Old style callback is deprecated.  See: {link}', UserWarning)


mse,61.64325
rmse,7.85132
_runtime,4076
_timestamp,1630301465
_step,3323


mse,▁
rmse,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 8n9np99w with config:
wandb: 	learning_rate: 0.06414550510806796
wandb: 	max_depth: 4
wandb: 	n_estimators: 2695
wandb: 	reg_alpha: 1.9359213929455321
wandb: 	reg_lambda: 2.9247059290587636
wandb: 	subsample: 0.7734314704689422
wandb: wandb version 0.12.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


...

/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/xgboost/training.py:17: UserWarning: Old style callback is deprecated.  See: https://xgboost.readthedocs.io/en/latest/python/callbacks.html
  warnings.warn(f'Old style callback is deprecated.  See: {link}', UserWarning)


mse,61.46125
rmse,7.83972
_runtime,234
_timestamp,1630337722
_step,715


mse,▁
rmse,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Agent Starting Run: 24hglhue with config:
wandb: 	learning_rate: 0.11789665750762816
wandb: 	max_depth: 2
wandb: 	n_estimators: 1048
wandb: 	reg_alpha: 2.137399071610554
wandb: 	reg_lambda: 2.139375668293327
wandb: 	subsample: 0.9712934256472006
wandb: wandb version 0.12.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/xgboost/training.py:17: UserWarning: Old style callback is deprecated.  See: https://xgboost.readthedocs.io/en/latest/python/callbacks.html
  warnings.warn(f'Old style callback is deprecated.  See: {link}', UserWarning)
wandb: Ctrl + C detected. Stopping sweep.
